In [1]:
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper
from torch.utils.data import DataLoader
from dataset.dataset_loader import dataset_loader

# What image size? Let's go for a common image size while allowing the network to be smaller:
# 160x160: A bit larger than 128x128, this size provides slightly more detail while still maintaining reasonable computational requirements. It's suitable for various tasks, including object detection and classification.
train, test = dataset_loader((160, 160), is_grayscale=False)

# We take 20% of dataset as validation.
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test, batch_size=32, shuffle=True)

In [2]:
print(f"Validation set is {len(validation) / (len(train) + len(test)) * 100:.2f}% of total dataset.")

In [3]:
from models.structure.learning_parameters.sgd_learning_parameters import SgdLearningParameters
from models.naive_dnn_gen.naive_dnn import NaiveDnnWrapper

# For input of size 160x160x3 (150528) we have the problem to choose the neurons between that and 1 (output).
# It would be reasonable to have large hidden layers. Yet we are limited.
model = NaiveDnnWrapper().make_model((3, 160, 160))

learning_parameters = SgdLearningParameters(learning_rate=1e-4)
learning_parameters.compile_model(model)

model.summary()

I don't know why but it considers 16 for batch size while I didn't explicit it.

In [4]:
import keras

keras.utils.plot_model(
    model, to_file='hand_tailored_conv_net.png', show_layer_names=True, expand_nested=True, show_shapes=True
)

In [5]:
import keras

# In this simple approach we have fixed [Train 0.8 - Val 0.2 - Test] without doing any k-fold CV.
# The number of epoches is fixed by early stopping. (In the case of k-fold-CV it will be a hp given by average performance?)
history = model.fit(x=train_dataloader, validation_data=validation_dataloader, batch_size=32, epochs=120, callbacks=[
    # To avoid going further when training
    keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=1e-4, patience=10,
        verbose=1, mode='min', restore_best_weights=True
    ),
    # To persist the history
    keras.callbacks.CSVLogger(f"hand-tailored-dnn.log", separator=",", append=True)
])

In [7]:
res = model.evaluate(test_dataloader)
persist_model: bool = True

# We don't want to persist the model as I already saved it.
if persist_model:
    model.save('hand_tailored_dnn_net.keras')

In [8]:
print(f"Test accuracy is {res[1] * 100:.2f}% while loss is {res[0]}")
print(f"The model is not that bad considering we have no pre-processing done and the parameters were chosen freely")

In [9]:
import pandas

csv = pandas.read_csv(f"hand-tailored-dnn.log")

In [13]:
import plotly.express as px

loss_figure = px.line(csv, x="epoch", y=["loss", "val_loss"], template="plotly_white", markers=True)
loss_figure.update_layout(title="Loss", xaxis_title="Epoch", yaxis_title="Loss")

In [12]:
figure = px.line(csv, x="epoch", y=["accuracy", "val_accuracy"], template="plotly_white", markers=True)
figure.update_layout(title="Accuracy", xaxis_title="Epoch", yaxis_title="Loss")

The model isn't necessarily evil. It works fair enough not being tuned.
Regularization and images pre-processing could lead to some good results while maybe it is the case to increase the parameters? All these considerations are to left to thought as the CNN (which is a state of the art approach) is only 1/3 of the current network and performs consistently better in the same amount of epochs on test while having more trouble fitting the training set.

### CNN seem to be the path to take (as tweaking a DNN can be a really hard task)